# Etapa 1: leitura do arquivo e criação do DataFrame
Para realizar a tarefa devemos realizar a leitura do arquivo. Para isso, utilizamos a função read_csv presente na biblioteca pandas.

In [21]:
import pandas as pd
import numpy as np

# realiza a leitura dos dados
dataset = pd.read_csv('../data/cars.csv')
# realizar a leitura dos valores de todas as colunas 
X = dataset.iloc[:,:-1].values
# transforma a matriz em um dataframe
X = pd.DataFrame(X)
# exibe os 20 primeiros registros do DataFrame
X.head(20)

,0,1,2,3,4,5,6
0,14,8,350,165,4209,12,1972
1,31.9,4,89,71,1925,14,1980
2,17,8,302,140,3449,11,1971
3,15,8,400,150,3761,10,1971
4,30.5,4,98,63,2051,17,1978
5,23,8,350,125,3900,17,1980
6,13,8,351,158,4363,13,1974
7,14,8,440,215,4312,9,1971
8,25.4,5,183,77,3530,20,1980
9,37.7,4,89,62,2050,17,1982


# Etapa 2: transformação dos dados

In [22]:
# lista com as colunas do DataFrame
columnlist = ['consumo', ' cilindros', ' cilindrada', 'potencia', ' peso', 'tempo-0-a-100', 'ano']
# atribui, a cada coluna, seu respectivo título através do atributo colmuns
X.columns = columnlist

# coluna a coluna, tranforma os dados faltantes em dados numéricos. Para isso usamos a função do pandas to_numeric
#    No nosso caso, fazemos o mais simples que é preenchê-los com 0.
for i in range(0, len(columnlist)):
    X[columnlist[i]] = pd.to_numeric(X[columnlist[i]], errors='coerce').fillna(0).astype(np.int64)

# exibimos novamente para verificar que os dados agora são numéricos
X.head(20)

,consumo,cilindros,cilindrada,potencia,peso,tempo-0-a-100,ano
0,14,8,350,165,4209,12,1972
1,31,4,89,71,1925,14,1980
2,17,8,302,140,3449,11,1971
3,15,8,400,150,3761,10,1971
4,30,4,98,63,2051,17,1978
5,23,8,350,125,3900,17,1980
6,13,8,351,158,4363,13,1974
7,14,8,440,215,4312,9,1971
8,25,5,183,77,3530,20,1980
9,37,4,89,62,2050,17,1982


# Etapa 3: Determinação do número ótimo de cluster

Escolhemos, arbitrariamente, uma quantidade de clusters que imaginamos podem ser agrupados nosso dataset. Rodamos, então, o KMeans para cada um desse valores identificando: a quantidade de iterações "max_iter", o número de seeds a cada nova rodada "n_init", e se queremos utilizar alguma função para geração das seeds "random_state"

Registramos, a cada iteração, a soma quadrática das distâncias de cada instância do cluster ao centróide (silhueta)



In [23]:
from sklearn.cluster import KMeans
# vetor de resultados
wcss = []
# escolhemos de 1 a 11 clusters para análise
for i in range(1,11):
    # criamos o modelo KMeans a ser testado
    kmeans = KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0)
    # rodamos o KMeans sobre a base de dados
    kmeans.fit(X)
    # avaliamos a qualidade da clusterização realizada
    wcss.append(kmeans.inertia_)

# Etapa 4: Avaliar os clusters pelo método Elbow
O gráfico demostra os resultados da análise dos clusters e permite escolher qual a quantidade de clusters que queremos usar

In [35]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

# criamos o gráfico com os dados coletados
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.show()

ValueError: matplotlib display text must have all code points < 128 or use Unicode strings

<Figure size 432x288 with 1 Axes>

# Etapa 5: Realizamos a clusterização
Realiza a clusterização com base na quantidade de clusters definidos

In [36]:
# definimos a quantidade de clusters ideal
clusters = 4
# Aplicamos o KMeans com a quantidade de clusters determinados
kmeans = KMeans(n_clusters=clusters,init='k-means++',max_iter=300,n_init=10,random_state=0) 
# realizamos a predição sobre a base de dados
y_kmeans = kmeans.fit_predict(X)

# Etapa 6: Plotamos o resultado
Avaliar os resultados não é trivial. Vamos plotar em duas dimensões então só podemos avaliar duas características por vez.

In [28]:
# para plotar os dados transformamos o DataFrame em uma matriz de valores numéricos 
R = X.as_matrix(columns=None)
# se quiser mais cores visite https://www.webucator.com/blog/2015/03/python-color-constants-module/
color=['red','green','blue','black','turquoise','violet','orange']
# determina quais as características serão plotadas no gráfico no caso a primeira coluna (0 - 'consumo') e a segunda (1 - 'cilindros')
x_axis = 0
y_axis = 1

# plota os clusters utilizando duas características do modelo
for i in range(0,clusters):
    labelCluster = 'Cluster '+str(i+1)
    plt.scatter(R[y_kmeans == i, x_axis], R[y_kmeans == i, y_axis],s=50, c=color[i],label=labelCluster)
# plota no gráfico
plt.scatter(kmeans.cluster_centers_[:,x_axis],kmeans.cluster_centers_[:,y_axis],s=300,c='yellow',label='Centroides')
plt.title('Clusterização dos modelos de carro')
plt.legend()
plt.show()

/home/alex/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


ValueError: matplotlib display text must have all code points < 128 or use Unicode strings

<Figure size 432x288 with 1 Axes>

# Exercício

O gráfico acima foi feito com 2 clusters e foi plotado com duas variáveis que não permitem visualizar como os clusters estão, de fato separado. Refaça a clusterização considerando o resultado obtido com o método Elbow e escolha as características que demonstrem a melhor separação no gráfico.
Relatório:
> Seu relatório final da prática deve conter a escolha, e sua justificativa, da quantidade de clusters e o gráfico que melhor demostra a separação dos grupos. 